### The `range` Function — Advanced Usage (compact & practical)

This notebook goes beyond basics: slicing `range` objects, membership math, length tricks, reversing, performance notes, and a few handy patterns.

Key idea: `range(start, stop, step)` is an **immutable, lazy** arithmetic progression that behaves like a read-only sequence (supports `len`, indexing, slicing, containment) without storing all values.

#### 1) Anatomy & properties

In [1]:
r = range(2, 11, 2)    # 2,4,6,8,10
type(r), r.start, r.stop, r.step, len(r), r[0], r[-1]

(range, 2, 11, 2, 5, 2, 10)

#### 2) Indexing & slicing (slicing returns **another range**)

Unlike lists, slicing a `range` yields a new `range`—still lazy and memory-cheap.

In [2]:
r = range(10)          # 0..9
r5 = r[::2]            # even indices from 0..9 → 0,2,4,6,8
r_slice = r[3:9:2]     # 3,5,7
r, r5, r_slice, type(r_slice), list(r_slice)

(range(0, 10), range(0, 10, 2), range(3, 9, 2), range, [3, 5, 7])

#### 3) Reversing a range (two equivalent ways)

In [3]:
r = range(1, 6)        # 1..5
rev1 = r[::-1]         # slicing
rev2 = range(r.stop-1, r.start-1, -r.step)  # explicit reverse
list(rev1), list(rev2)

([5, 4, 3, 2, 1], [5, 4, 3, 2, 1])

#### 4) Membership (`in`) is O(1) via arithmetic, not search

An integer `x` is in `range(a,b,s)` iff `a ≤ x < b` (or `a ≥ x > b` for negative `s`) **and** `(x - a)` is divisible by `s`.

In [4]:
r = range(3, 20, 4)    # 3,7,11,15,19
tests = [3, 4, 11, 18, 19, 23]
{x: (x in r) for x in tests}

{3: True, 4: False, 11: True, 18: False, 19: True, 23: False}

#### 5) Length formula (works for both positive and negative steps)

Python uses a safe arithmetic formula; you can mirror it if needed:

In [5]:
def range_len(start, stop, step=1):
    if step == 0:
        raise ValueError('step must not be zero')
    # Align direction
    if (step > 0 and start >= stop) or (step < 0 and start <= stop):
        return 0
    # Ceil division for ints with sign-aware math
    return (abs(stop - start) + abs(step) - 1) // abs(step)

cases = [(0,10,1), (2,11,2), (10,2,-2), (5,5,1), (5,5,-1)]
{c: range_len(*c) for c in cases}, {c: len(range(*c)) for c in cases}

({(0, 10, 1): 10, (2, 11, 2): 5, (10, 2, -2): 4, (5, 5, 1): 0, (5, 5, -1): 0},
 {(0, 10, 1): 10, (2, 11, 2): 5, (10, 2, -2): 4, (5, 5, 1): 0, (5, 5, -1): 0})

#### 6) `.index` and `.count` on `range`

`range` implements sequence methods: `x in r`, `r.index(x)`, `r.count(x)` (no scanning of all elements).

In [6]:
r = range(10, 21, 5)   # 10,15,20
r.count(15), r.count(16), r.index(20)

(1, 0, 2)

#### 7) Large ranges are tiny in memory (lazy)

In [7]:
import sys
huge = range(0, 10**12, 10)
sys.getsizeof(huge)  # small, independent of length

48

#### 8) Using `range` for indices, batching, and pairing

- Prefer `for i, x in enumerate(seq)` to `for i in range(len(seq))` when you need values.
- For **windows/batches**, step the index.
- Pair multiple ranges with `zip`.

In [8]:
data = list('abcdefg')
# Enumerate indices + values
pairs = [(i, ch) for i, ch in enumerate(data)]

# Fixed-size batches (size=3)
size = 3
batches = [data[i:i+size] for i in range(0, len(data), size)]

# Zipping ranges (parallel progressions)
r1 = range(0, 10, 2)
r2 = range(100, 110, 2)
paired = list(zip(r1, r2))
pairs, batches, paired[:3]

([(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f'), (6, 'g')],
 [['a', 'b', 'c'], ['d', 'e', 'f'], ['g']],
 [(0, 100), (2, 102), (4, 104)])

#### 9) Edge cases & gotchas

- `step` cannot be `0`.
- Direction matters: with negative `step`, `start` should exceed `stop` or length becomes `0`.
- Slicing composes steps: `range(...)[a:b:c]` multiplies step effectively (and still returns a `range`).
- Membership is arithmetic; floating points are **not** allowed (only ints).

#### 10) Practical mini-exercises (quick checks)

In [9]:
# 10.1) All multiples of 7 between 50 and 200 inclusive, lazily
first = 50 + ((7 - 50 % 7) % 7)
r7 = range(first, 201, 7)
len(r7), r7.start, r7.stop, r7.step, list(r7[:5])  # peek

# 10.2) Every 3rd element of a reversed 1..100 sequence (i.e., 100,97,94,...)
rev3 = range(100, 0, -3)
rev3[:10]

# 10.3) Work with slices without materializing: (range(0,50)[5:40:5])
sub = range(0, 50)[5:40:5]
sub, list(sub)

(range(5, 40, 5), [5, 10, 15, 20, 25, 30, 35])